## Setup Connection to Graph Database

Download Neo4j locally [here](https://neo4j.com/download/?ref=docs-get-started-dropdown). The default database is called `neo4j`.

In [33]:
import neo4j
import os, shutil
import pandas as pd
import tqdm

In [37]:
driver = neo4j.GraphDatabase.driver("neo4j://localhost", auth=('neo4j', 'password'))
driver.verify_connectivity()

## Load CSV of translated DRKG relationships

The data file must be in the Neo4j import directory. See [here](https://neo4j.com/docs/operations-manual/5/configuration/file-locations/) for the import directory's location on your system.

In [11]:
# edit the below according to your Neo4j install
NEO4J_HOME = "/home/alexdloia/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-32f29f82-74c3-4124-8a29-27ad205fb889"
drkg_csv_name = 'drkg_translated_with_relation_labels.csv'
drkg_data_path = os.path.join('..', 'data', drkg_csv_name)

In [ ]:
shutil.copy(drkg_data_path, os.path.join(NEO4J_HOME, 'import', drkg_csv_name))

In [15]:
drkg_df = pd.read_csv(drkg_data_path, index_col="id")
drkg_df.head()

,head_entity,tail_entity,drkg_id,relation_label,head_entity_type,tail_entity_type
id,,,,,,
0,coagulation factor VIII (F8),coagulation factor VIII (F8),bioarx::HumGenHumGen:Gene:Gene,INTERACTION,GENE,GENE
1,coagulation factor VIII (F8),phytanoyl-CoA 2-hydroxylase (PHYH),bioarx::HumGenHumGen:Gene:Gene,INTERACTION,GENE,GENE
2,coagulation factor VIII (F8),coagulation factor IX (F9),bioarx::HumGenHumGen:Gene:Gene,INTERACTION,GENE,GENE
3,coagulation factor VIII (F8),heat shock protein family A (Hsp70) member 5 (...,bioarx::HumGenHumGen:Gene:Gene,INTERACTION,GENE,GENE
4,coagulation factor VIII (F8),immunoglobulin kappa variable 3-20 (IGKV3-20),bioarx::HumGenHumGen:Gene:Gene,INTERACTION,GENE,GENE


In [18]:
for col_name in ["head_entity_type", "tail_entity_type"]:
    drkg_df[col_name] = drkg_df[col_name].str.replace(" ", "_")

In [47]:
unique_entity_types = pd.concat(
    [drkg_df["head_entity_type"], drkg_df["tail_entity_type"]]
).unique()
unique_relation_labels = drkg_df["relation_label"].unique()
unique_entity_types, unique_relation_labels

(array(['GENE', 'COMPOUND', 'DISEASE', 'ANATOMY', 'PHARMACOLOGIC_CLASS',
        'BIOLOGICAL_PROCESS', 'SYMPTOM', 'MOLECULAR_FUNCTION',
        'CELLULAR_COMPONENT', 'PATHWAY', 'SIDE_EFFECT'], dtype=object),
 array(['INTERACTION', 'INHIBITION', 'OTHER', 'ACTIVATION', 'BINDING',
        'MODULATION', 'BLOCKING', 'ANTAGONISM', 'ANTIBODY',
        'POSITIVE_ALLOSTERIC_MODULATION', 'ALLOSTERIC_MODULATION',
        'PARTIAL_AGONISM', 'DRUGDRUG_INTERACTION', 'TARGET', 'ENZYME',
        'CARRIER', 'COMPOUND_TREATS_THE_DISEASE',
        'AFFECTS_EXPRESSION_ORPRODUCTION_NEUTRAL',
        'METABOLISM_OR_PHARMACOKINETICS',
        'DECREASES_EXPRESSION_ORPRODUCTION',
        'INCREASES_EXPRESSION_ORPRODUCTION', 'TRANSPORT_OR_CHANNELS',
        'INHIBITS_CELL_GROWTH_ESPECIALLY_CANCERS',
        'SIDE_EFFECT_ORADVERSE_EVENT', 'BIOMARKERS_OF_DISEASE_PROGRESSION',
        'ALLEVIATES_OR_REDUCES', 'PREVENTS_OR_SUPPRESSES',
        'ROLE_IN_DISEASE_PATHOGENESIS',
        'IMPROPER_REGULATION_LINKED_TO_

In [30]:
entity_type_pairs = pd.concat(
    [
        drkg_df[["head_entity", "head_entity_type"]].rename(
            columns={"head_entity": "entity", "head_entity_type": "type"}
        ),
        drkg_df[["tail_entity", "tail_entity_type"]].rename(
            columns={"tail_entity": "entity", "tail_entity_type": "type"}
        ),
    ],
    axis=0,
    ignore_index=True,
).groupby(["entity", "type"]).size().reset_index()
entity_type_pairs.drop(columns=0, inplace=True)
entity_type_pairs.rename(columns={'entity': 'name'}, inplace=True)
entity_type_pairs.head()

,name,type
0,'de novo' AMP biosynthetic process,BIOLOGICAL_PROCESS
1,'de novo' CTP biosynthetic process,BIOLOGICAL_PROCESS
2,'de novo' GDP-L-fucose biosynthetic process,BIOLOGICAL_PROCESS
3,'de novo' IMP biosynthetic process,BIOLOGICAL_PROCESS
4,'de novo' NAD biosynthetic process,BIOLOGICAL_PROCESS


In [31]:
entity_type_pairs_l = entity_type_pairs.to_dict('records')
entity_type_pairs_l[:5]

[{'name': "'de novo' AMP biosynthetic process", 'type': 'BIOLOGICAL_PROCESS'},
 {'name': "'de novo' CTP biosynthetic process", 'type': 'BIOLOGICAL_PROCESS'},
 {'name': "'de novo' GDP-L-fucose biosynthetic process",
  'type': 'BIOLOGICAL_PROCESS'},
 {'name': "'de novo' IMP biosynthetic process", 'type': 'BIOLOGICAL_PROCESS'},
 {'name': "'de novo' NAD biosynthetic process", 'type': 'BIOLOGICAL_PROCESS'}]

In [32]:
records, summary, keys = driver.execute_query(
    "UNWIND $props AS map\n" + "CREATE (n:Entity)\n" + "SET n = map",
    props=entity_type_pairs_l,
)

In [41]:
summary.counters

{'labels_added': 62374, 'nodes_created': 62374, 'properties_set': 124748}

In [34]:
drkg_df.loc[0]

head_entity           coagulation factor VIII (F8)
tail_entity           coagulation factor VIII (F8)
drkg_id             bioarx::HumGenHumGen:Gene:Gene
relation_label                         INTERACTION
head_entity_type                              GENE
tail_entity_type                              GENE
Name: 0, dtype: object

In [46]:
drkg_df.index[drkg_df["relation_label"] == "ACTIVATION"]

Index([  77274,   77321,   77341,   77344,   77346,   77350,   77360,   77363,
         77370,   77399,
       ...
       1538633, 1538711, 1538797, 1538800, 1538874, 1538888, 1538943, 1538970,
       1539019, 1539030],
      dtype='int64', name='id', length=3234)

In [49]:
driver.execute_query("CREATE INDEX entity_name_type FOR (e:Entity) ON (e.name, e.type)")

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7fe2d6c095d0>, keys=[])

In [50]:
for relation_label in unique_relation_labels:
    # create relationships for each label according to drkg_df
    this_index = drkg_df.index[drkg_df["relation_label"] == relation_label]
    if relation_label == "INTERACTION":
        this_index = this_index[82023:]
    for i in tqdm.tqdm(this_index):
        row = drkg_df.loc[i]
        rel_type = row["relation_label"]
        driver.execute_query(
            f"""
        MATCH 
            (e1:Entity {{name: $head_name, type: $head_type}}),
            (e2:Entity {{name: $tail_name, type: $tail_type}})
        CREATE
            (e1)-[r:{rel_type} {{type: $drkg_id}}]->(e2)
        """,
            head_name=row["head_entity"],
            tail_name=row["tail_entity"],
            head_type=row["head_entity_type"],
            tail_type=row["tail_entity_type"],
            drkg_id=row["drkg_id"],
            database_='neo4j'
        )

100%|██████████| 15113/15113 [00:56<00:00, 266.62it/s]
